In [31]:
import requests
import re
import math
import easier as ezr
import itertools
from pprint import pprint
%config Completer.use_jedi = False

In [41]:
class GithubBase:
    BASE_URL = 'https://api.github.com/gists'
    USER_URL = 'https://api.github.com/users/robdmc/gists'
    REX_NEXT = re.compile(r'.*<(https.*?)>; rel="next"')
    PER_PAGE = 99
#     PER_PAGE = 9
    MAX_GISTS = 10_000

    def __init__(self):
        self.access_token = "enter_token_here"

#     def gist_list(self):
#         # Initalize to empty blobs
#         blobs = []

#         # Get the first response
#         params = {
#             'access_token': self.access_token,
#             'per_page': self.PER_PAGE,
#             'page': 1
#         }
#         resp = requests.get(self.USER_URL, params=params)
#         return resp
#         # Process the first response
#         next_link, blobs = self.process_response(resp, blobs)
#         return blobs

#         # Process additional pages
#         while next_link is not None:
#             resp = requests.get(next_link)
#             next_link, blobs = self.process_response(resp, blobs)

#         return blobs

    def blob_generator(self):
        for page in range(int(math.ceil(self.MAX_GISTS / self.PER_PAGE))):
            params = {
                'per_page': self.PER_PAGE,
                'page': page,
            }
            headers = {
                'Authorization': f'token {self.access_token}',
                'accept': 'application/vnd.github.v3+json'
            }
            resp = requests.get(self.USER_URL, params=params, headers=headers)
            self.latest_resp = resp
            blobs = resp.json()
            if blobs:
                yield blobs
            else:
                break
                
    def get_list(self):
        rec_list = []
        for ll in self.blob_generator():
            rec_list.extend(ll)
        return rec_list
    
    @ezr.pickle_cached_container()
    def ll(self):
        return self.get_list()
    
    def get_parsed_blob_list(self):
        keys = [
            'url',
            'description',
            'created_at',
            'updated_at',

        ]
        records = []
        
        
        # for item in self.blob_generator():
        for item in self.ll:
            record_template = {k: item[k] for k in keys}
            for file in item['files'].values():
                record = record_template.copy()
        #         print(file)
                record['file'] = file['filename']
                record['language'] = file.get('language', 'unkown')
                record['file_url'] = file['raw_url']
                record['size'] = file.get('size', 0)
                records.append(record)
        return records



gh = GithubBase()
recs = gh.get_parsed_blob_list()
print(type(recs))
print(len(recs))

<class 'list'>
182


In [42]:
recs

[{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
  'description': 'Visidata Monstrosity',
  'created_at': '2021-01-17T17:14:45Z',
  'updated_at': '2021-01-17T19:55:10Z',
  'file': 'monstrous_visidatarc.py',
  'language': 'Python',
  'file_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
  'size': 1999},
 {'url': 'https://api.github.com/gists/3f82bc492e87a1d5955e43250b32051c',
  'description': '',
  'created_at': '2020-09-28T22:23:21Z',
  'updated_at': '2020-09-28T22:23:22Z',
  'file': 'networkx_plot_graphviz.py',
  'language': 'Python',
  'file_url': 'https://gist.githubusercontent.com/robdmc/3f82bc492e87a1d5955e43250b32051c/raw/01f3ef69e4d9c3769ea2cd45951dabb69650a284/networkx_plot_graphviz.py',
  'size': 151},
 {'url': 'https://api.github.com/gists/32275939f121fc7f5661967cc321527f',
  'description': 'Altair example',
  'created_at': '2020-06-16T23:20:11Z',


In [11]:
type(resp.headers)

requests.structures.CaseInsensitiveDict

In [37]:
blob = resp.json()

keys = [
    'url',
    'description',
    'created_at',
    'updated_at',
    
]
records = []
for item in blob:
    record_template = {k: item[k] for k in keys}
    for file in item['files'].values():
        record = record_template.copy()
#         print(file)
        record['file'] = file['filename']
        record['language'] = file.get('language', 'unkown')
        record['file_url'] = file['raw_url']
        record['size'] = file.get('size', 0)
        records.append(record)
#     pprint(item)

rec = records[0]
rec


{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
 'description': 'Visidata Monstrosity',
 'created_at': '2021-01-17T17:14:45Z',
 'updated_at': '2021-01-17T19:55:10Z',
 'file': 'monstrous_visidatarc.py',
 'language': 'Python',
 'file_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
 'size': 1999}

In [16]:
blob[0]

{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
 'forks_url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84/forks',
 'commits_url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84/commits',
 'id': '611b57470ca1522c4f3c496a7c967a84',
 'node_id': 'MDQ6R2lzdDYxMWI1NzQ3MGNhMTUyMmM0ZjNjNDk2YTdjOTY3YTg0',
 'git_pull_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84.git',
 'git_push_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84.git',
 'html_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84',
 'files': {'monstrous_visidatarc.py': {'filename': 'monstrous_visidatarc.py',
   'type': 'application/x-python',
   'language': 'Python',
   'raw_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
   'size': 1999}},
 'public': True,
 'created_at': '2021-01-17T17:14:45Z',
 'updated_at': '2021-01-17T19:55: